# 02 - SageMaker Training Job

This notebook:
- Configures SageMaker
- Trains built-in XGBoost model
- Saves trained model artifact
- Deploys trained model
- Generates predictions
- Creates Kaggle submission
- Deletes endpoint to stop billing


In [1]:
import sagemaker
from sagemaker.estimator import Estimator
import boto3

sess = sagemaker.Session()
region = sess.boto_region_name
role = sagemaker.get_execution_role()

bucket = sess.default_bucket()
prefix = "house-price-kaggle-xgb"


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [7]:
from sagemaker.image_uris import retrieve
container = retrieve("xgboost", region=region, version="1.7-1")
print(container)



INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1


In [3]:
xgb = Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size=5,
    max_run=3600,
    output_path=f"s3://{bucket}/{prefix}/output",
    sagemaker_session=sess
)


In [4]:
xgb.set_hyperparameters(
    objective="reg:squarederror",
    eval_metric="rmse",
    num_round=2000,
    eta=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8
)


In [10]:
from sagemaker.inputs import TrainingInput

train_input = f"s3://{bucket}/{prefix}/data/train.csv"
val_input   = f"s3://{bucket}/{prefix}/data/validation.csv"

inputs = {
    "train": TrainingInput(train_input, content_type="text/csv"),
    "validation": TrainingInput(val_input, content_type="text/csv"),
}


In [11]:
xgb.fit(inputs)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2026-02-12-19-02-39-467


2026-02-12 19:02:42 Starting - Starting the training job...
2026-02-12 19:02:57 Starting - Preparing the instances for training...
2026-02-12 19:03:24 Downloading - Downloading input data...
2026-02-12 19:04:04 Downloading - Downloading the training image......
2026-02-12 19:05:00 Training - Training image download completed. Training in progress./miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2026-02-12 19:05:02.928 ip-10-2-219-33.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2026-02-12 19:05:02.965 ip-10-2-219-33.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2026-02-12:19:05:03:INFO] Imported framework sagemaker_xgboost_container.training
[2026-02-12:19:05:03:INFO] Failed to parse hyperparameter eval_metric value rmse 

In [12]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import StringDeserializer

predictor = xgb.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    serializer=CSVSerializer(),
    deserializer=StringDeserializer()
)

print("Endpoint deployed.")


INFO:sagemaker:Creating model with name: sagemaker-xgboost-2026-02-12-20-41-12-699
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2026-02-12-20-41-12-699
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2026-02-12-20-41-12-699


------!Endpoint deployed.


In [18]:
import re
import numpy as np

def predict_in_batches(predictor, X, batch_size=200):
    preds = []
    for start in range(0, len(X), batch_size):
        end = start + batch_size
        payload = X.iloc[start:end].to_csv(index=False, header=False)
        out = predictor.predict(payload)

        # out is a string like: "0.12\n0.34\n..." OR "0.12,0.34,..."
        out = out.strip()

        # Extract all floats robustly
        numbers = re.findall(r"[-+]?\d*\.\d+|\d+", out)
        batch_preds = np.array(numbers, dtype=np.float32)

        preds.append(batch_preds)

    return np.concatenate(preds) if len(preds) else np.array([])



In [19]:
import pandas as pd
import numpy as np

X_test = pd.read_csv("../processed/test.csv", header=None)
X_test = X_test.astype(np.float32)

test_pred_log = predict_in_batches(predictor, X_test, batch_size=200)

print("Pred count:", len(test_pred_log), "Expected:", len(X_test))

test_pred_price = np.expm1(test_pred_log)

test_df = pd.read_csv("../data/test.csv")

submission = pd.DataFrame({
    "Id": test_df["Id"].values,
    "SalePrice": test_pred_price
})

submission.to_csv("../submission.csv", index=False)
print("✅ Saved: ../submission.csv")
submission.head()


Pred count: 1459 Expected: 1459
✅ Saved: ../submission.csv


,Id,SalePrice
0,1461,111125.703125
1,1462,137742.687500
2,1463,167518.546875
3,1464,174800.171875
4,1465,163386.750000


In [20]:
predictor.delete_endpoint()
print("Endpoint deleted (billing stopped).")


INFO:sagemaker:Deleting endpoint configuration with name: sagemaker-xgboost-2026-02-12-20-41-12-699
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2026-02-12-20-41-12-699


Endpoint deleted (billing stopped).
